## Read Khan Academy Lessons Mongo Collection and generate Word Templates

In [26]:
import os
import pymongo
import pandas as pd
import re

In [28]:
# Script Variables

subject = 'High school statistics'
# subject = 'Statistics and probability'
# subject = 'AP®︎/College Statistics'


In [73]:
# Connect to MongoDB and load lessons collection into Pandas Dataframe

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['khan-academy']

# Lessons collection
colLessons = db["lessons"]

query = { 'subject': subject, 'lesson_type': "Article", 'lesson_type': "Video" }
# query = { 'subject': subject, 'lesson_type': { $in: ["Video", "Article"] } }
# { $in: [ "ALGOL", "Lisp" ]} 
# columns = {'_id': 0, 'unit_sequence': 0, 'lesson_type': 0, 'lesson_link': 0 }
columns = {'_id': 0, 'unit_sequence': 0, 'lesson_link': 0 }


# Ver
# https://www.mongodb.com/docs/manual/reference/method/db.collection.find/



# lessons = colLessons.find(query, columns)
lessons = db.lessons.find(query, columns)

list_lessons = list(lessons)
df = pd.DataFrame(list_lessons)

df.head()

,subject,unit,group,lesson,lesson_type
0,High school statistics,Unit 1: Displaying a single quantitative variable,Frequency tables and dot plots,Frequency tables & dot plots,Video
1,High school statistics,Unit 1: Displaying a single quantitative variable,Histograms,Creating a histogram,Video
2,High school statistics,Unit 1: Displaying a single quantitative variable,Histograms,Interpreting a histogram,Video
3,High school statistics,Unit 1: Displaying a single quantitative variable,Mean and median in data displays,"Statistics intro: Mean, median, & mode",Video
4,High school statistics,Unit 1: Displaying a single quantitative variable,Mean and median in data displays,Median in a histogram,Video


In [68]:
df.index

RangeIndex(start=0, stop=99, step=1)

In [ ]:
# https://towardsdatascience.com/10-python-pandas-tricks-to-make-data-analysis-more-enjoyable-cb8f55af8c30
# https://pandas.pydata.org/docs/user_guide/style.html

# df.head(20).style.set_table_styles(
# [{'selector': 'th',
#   'props': [('background', '#7CAE00'), 
#             ('color', 'white'),
#             ('font-family', 'verdana')]},
 
#  {'selector': 'td',
#   'props': [('font-family', 'verdana')]},

#  {'selector': 'tr:nth-of-type(odd)',
#   'props': [('background', '#DCDCDC')]}, 
 
#  {'selector': 'tr:nth-of-type(even)',
#   'props': [('background', 'white')]},
 
# ]
# ).hide(axis="index")